# descargas

> Funciones para descargar, procesar y actualizar las fuentes de datos que se usan en la librería. 

In [ ]:
#| default_exp descargas

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import pandas as pd
import geopandas as gpd
import requests
import xml.etree.ElementTree as ET
import os
from zipfile import ZipFile
import shutil
from pathlib import Path

## descarga_denue

Esta función descarga diferentes bases preprocesadas del DENUE. En general el procesamiento involucra integrar los sectores en una base única y homogeneizar los nombres de los campos.

La función admite un parámetro que controla cuál base se descarga, toma en cuenta que la base completa es MUY grande.


In [ ]:
#| export
def descarga_denue(
        tipo:str='zmvm_2022' # Qué archivo vamos a descargar: ejemplo, zmvm_2022, zmvm_full, mexico_2022, mexico_full
    ) -> None:
    DOWNLOADS_PATH = "datos/descargas/"
    Path(DOWNLOADS_PATH).mkdir(parents=True, exist_ok=True)
    if tipo == 'zmvm_2022':
        fname = 'denue_2022_zmvm.gpkg'
        url = 'https://www.dropbox.com/s/cxyekc0wzzq6pc4/denue_2022_zmvm.gpkg?dl=1'
    elif tipo == 'ejemplo':
        fname = 'ejemplo_denue.gpkg'
        url = 'https://www.dropbox.com/s/670yx6un0t4sqal/ejemplo_denue.gpkg?dl=1'
    else:
        print(f'El tipo {tipo} todavía no está implementado')
        return

    if os.path.exists(DOWNLOADS_PATH + fname):
        print("El archivo ya está descargado")
    else:
        r = requests.get(url, allow_redirects=True)
        open(DOWNLOADS_PATH + fname, 'wb').write(r.content)
    


In [ ]:
descarga_denue()
descarga_denue('ejemplo')

El archivo ya está descargado
El tipo ejemplo todavía no está implementado


UnboundLocalError: local variable 'fname' referenced before assignment

## descarga_redes

Esta función descarga un Geopackage con la red de transporte importada de [OpenStreetMap](https://www.openstreetmap.org/#map=5/23.944/-102.579).

**Nota:** En este momento la red sólo está disponible para la Zona Metropolitana de la Ciudad de México.

In [ ]:
#| export
def descarga_redes():
    DOWNLOADS_PATH = "datos/descargas/"
    Path(DOWNLOADS_PATH).mkdir(parents=True, exist_ok=True)
    fname = 'red_zmvm.gpkg'
    if os.path.exists(DOWNLOADS_PATH + fname):
        print("El archivo ya está descargado")
    else:
        url = 'https://www.dropbox.com/s/0fq8e8v2axyxxoc/red_zmvm.gpkg?dl=1'
        r = requests.get(url, allow_redirects=True)
        open(DOWNLOADS_PATH + fname, 'wb').write(r.content)

In [ ]:
descarga_redes()

El archivo ya está descargado


## descarga_poligonos_ciudades

Esta función descarga las slhuetas de las 100 ciudades más grandes de México (SUN, 2016)

In [ ]:
#| export
def descarga_poligonos_ciudades():
    DOWNLOADS_PATH = "datos/descargas/"
    Path(DOWNLOADS_PATH).mkdir(parents=True, exist_ok=True)
    fname = 'silhuetas.shp.zip'
    if os.path.exists(DOWNLOADS_PATH + fname):
        print("El archivo ya está descargado")
    else:
        url = 'https://www.dropbox.com/s/kofmn5qws911ktg/silhueta_100_ciudades.zip?dl=1'
        r = requests.get(url, allow_redirects=True)
        open(DOWNLOADS_PATH + fname, 'wb').write(r.content)

In [ ]:
descarga_poligonos_ciudades()

## descarga_datos_completos

Esta función llama a todas las anmteriores para descargar todos los datos.

In [ ]:
#| export
def descarga_datos_completos():
    descarga_poligonos_ciudades()
    descarga_redes()

descarga_datos_completos()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()